In [ ]:
from google.colab import drive, runtime
drive.mount('/content/drive')

In [ ]:
!mkdir /content/data
!curl -L -o /content/data/stock-market-data-nifty-100-stocks-5-min-data.zip\
  https://www.kaggle.com/api/v1/datasets/download/debashis74017/stock-market-data-nifty-100-stocks-5-min-data
!unzip /content/data/stock-market-data-nifty-100-stocks-5-min-data.zip -d /content/data/ > /dev/null
!git clone https://github.com/erfan-bayat13/chronos-forecasting.git
%cd chronos-forecasting/
!pip install -e /content/chronos-forecasting > /dev/null
!pip install gluonts typer accelerate typer_config triton datasets > /dev/null

Create the datasets

In [ ]:
%run Ex_1_financial_finetuning/mixup.py
%run Ex_1_financial_finetuning/GBM_generation.py --seed 42 --no-savefig

Train the model. This should be done twice, tweaking the `train_financial_mixup.yaml` file in order to change the percentages between real and synthetic data (once with 90% and 10% and the other one with 100% and 0%).

In [ ]:
!python scripts/training/train.py --config Ex_1_financial_finetuning/configs/train_financial_mixup.yaml

Optional: visualize training info with Tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/Financial\ Extension/New\ Dataset/run-0/logs

Evaluate the model. This cell should be run multiple times editing the file `eval_finance_dataset.yaml` changing the offset and the prediction_length. Moreover, one should change the file names each timein order to match this (e.g.


In [ ]:
!mkdir -p /content/drive/MyDrive/Financial\ Extension/New\ Dataset/Results
!python Ex_1_financial_finetuning/custom_evaluation.py Ex_1_financial_finetuning/configs/eval_finance_dataset.yaml  /content/drive/MyDrive/Financial\ Extension/New\ Dataset/Results/results_before_finetuning_96_pl.csv --chronos-model-id amazon/chronos-t5-tiny
!python Ex_1_financial_finetuning/custom_evaluation.py Ex_1_financial_finetuning/configs/eval_finance_dataset.yaml  /content/drive/MyDrive/Financial\ Extension/New\ Dataset/Results/results_after_finetuning_10_perc_96_pl.csv --chronos-model-id /content/drive/MyDrive/Financial\ Extension/New\ Dataset/run-0/checkpoint-final

Make and save plots

In [ ]:
%run /content/chronos-forecasting/Ex_1_financial_finetuning/make_plots.py
%run Ex_1_financial_finetuning/GBM_generation.py --display-first 20 --output-dir /content/data/GBM_synth \
                      --num-series 5000 --seed 42 --steps-in-year 105120 --no-save --savefig --min-initial-price 300 --max-initial-price 300